In [ ]:
import os
import pandas as pd



In [ ]:
# Excel dosyasını pandas DataFrame'e yükle
df = pd.read_excel('./data_without_extra_features/SEFACED.xlsx')

# DataFrame'i bir CSV dosyasına kaydet
df.to_csv('./data_without_extra_features/SEFACED.csv', index=False)  # İndeks sütunu olarak satır numaralarının kaydedilmesini önlemek için index=False olarak ayarlayın



In [ ]:
# 9000 normal, 9000 fraudulent

# CSV dosyasını pandas DataFrame'e yükle
df = pd.read_csv('./data_without_extra_features/SEFACED.csv')


# 'Normal' etiketine sahip satırları filtrele ve ilk 4000 satırı tut
normal_rows = df[df['Class_Label'] == 'Normal']


# 'Fraudulent' etiketine sahip satırları filtrele ve ilk satırları tut
fraudulent_rows = df[df['Class_Label'] == 'Fraudulent']

# Filtrelenmiş DataFrame'leri birleştir
filtered_df = pd.concat([normal_rows, fraudulent_rows])


# Filtrelenmiş DataFrame'i yeni bir CSV dosyasına kaydet
filtered_df.to_csv('./data_without_extra_features/SEFACED_9000.csv', index=False)

In [ ]:
df1 = pd.read_csv("./data_without_extra_features/SEFACED_9000.csv")

In [ ]:
print(df1.head())

print(df1.info())

print(df1.describe())

                                                Text Class_Label
0  Let me know when you get the quotes from Pauli...      Normal
1  ---------------------- Forwarded by Phillip K ...      Normal
2  Steve, Please remove Bob Shiring and Liz River...      Normal
3  Go ahead and order the ac for #27. Can you ema...      Normal
4  Anymore details? Is the offer above or below 6...      Normal
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17640 entries, 0 to 17639
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Text         17541 non-null  object
 1   Class_Label  17640 non-null  object
dtypes: object(2)
memory usage: 275.8+ KB
None
                                                    Text Class_Label
count                                              17541       17640
unique                                             10670           2
top     <html><head><style>P{margin:0px;padding:0px}body      Normal
freq           

In [ ]:


import re


def load_stopwords(stopwords_file):
    with open(stopwords_file, 'r') as file:
        stopwords = file.read().splitlines()
    return stopwords


def clean_text(text, stopwords):
    if isinstance(text, str):
        # Küçük harfe çevir
        lowercase_text = text.lower()

        # Bağlantıları kaldır
        text_no_links = re.sub(r'http\S+|www.\S+|\.com|\w+@\w+\.\w+', '', lowercase_text)

        # HTML etiketlerini kaldır
        clean_html = re.sub(r'<.*?>', '', text_no_links)

        # Rakamları kaldır
        text_no_digits = re.sub(r'\d+', '', clean_html)

        # Özel karakterleri kaldır
        text_no_special_chars = re.sub(r'[^a-zA-Z0-9\s]', '', text_no_digits)

        # Metni tokenlara ayır
        tokens = text_no_special_chars.split()

        # Stop kelimelerini kaldır
        filtered_tokens = [token for token in tokens if token not in stopwords]

        # Tokenları tekrar metne birleştir
        clean_text = ' '.join(filtered_tokens)

        return clean_text

def clean_csv(input_csv, output_csv, stopwords_file):
    df = pd.read_csv(input_csv)
    stopwords = load_stopwords(stopwords_file)

    # 'Text' sütununu temizle
    df['cleaned_text'] = df['Text'].apply(lambda x: clean_text(x, stopwords))
    df = df.drop('Text', axis=1)

    # Temizlenmiş veriyi yeni bir CSV dosyasına kaydet
    df.to_csv(output_csv, index=False)

input_csv_file = './data_without_extra_features/SEFACED_9000.csv'
output_csv_file = './data_without_extra_features/preprocessed_SEFACED.csv' 
stopwords_file = './stopwords.txt'

clean_csv(input_csv_file, output_csv_file, stopwords_file)


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize


def tokenize_text(text):
    # Metnin bir dize olup olmadığını kontrol et veya dizeye dönüştür
    if isinstance(text, str):
        tokens = word_tokenize(text)
        return tokens
    else:
        # Girişin bir dize olmadığı durumda boş bir liste döndür
        return []

def tokenize_csv(input_csv, output_csv):
    # CSV dosyasını oku
    df = pd.read_csv(input_csv)

    # 'cleaned_text' sütununu tokenle
    df['tokens'] = df['cleaned_text'].apply(tokenize_text)
    df = df.drop('cleaned_text', axis=1)

    # Tokenleştirilmiş veriyi yeni bir CSV dosyasına kaydet
    df.to_csv(output_csv, index=False)

input_csv_file = './data_without_extra_features/preprocessed_SEFACED.csv'  
output_csv_file = './data_without_extra_features/tokenized_SEFACED.csv' 

tokenize_csv(input_csv_file, output_csv_file)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orange\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import words
nltk.download('wordnet')
nltk.download('words')

from nltk.stem import WordNetLemmatizer
import ast

# İngilizce kelime dağarcığını yükle
english_words = set(words.words())

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

def remove_misspelled(tokens):
    return [token for token in tokens if token in english_words]

def lemmatize_csv(input_csv, output_csv):
    # CSV dosyasını oku
    df = pd.read_csv(input_csv)

    # 'tokens' sütununu dizeden liste haline dönüştür
    df['tokens'] = df['tokens'].apply(ast.literal_eval)

    # 'tokens' sütununu lemmatize et
    df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_tokens)

    # Yanlış yazılmış kelimeleri kaldır
    df['lemmatized_tokens'] = df['lemmatized_tokens'].apply(remove_misspelled)

    # Orijinal 'tokens' sütununu kaldır
    df = df.drop('tokens', axis=1)

    # Lemmatize edilmiş veriyi yeni bir CSV dosyasına kaydet
    df.to_csv(output_csv, index=False)

# Kullanım örneği
input_csv_file = './data_without_extra_features/tokenized_SEFACED.csv'
output_csv_file = './data_without_extra_features/lemmatized_and_misspelled_removed_SEFACED.csv'
lemmatize_csv(input_csv_file, output_csv_file)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\orange\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\orange\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Ön işlenmiş veriyi oku
df = pd.read_csv('./data_without_extra_features/lemmatized_and_misspelled_removed_SEFACED.csv', encoding='utf-8')

# Lemmatize edilmiş tokenları ayıkla
corpus = df['lemmatized_tokens']

# Bir CountVectorizer oluştur
vectorizer = CountVectorizer()

# Corpus'u uyum sağla ve dönüştür, seyrek matrise
X_bow = vectorizer.fit_transform(corpus)

# Seyrek matrisi bir DataFrame'e dönüştür
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

print(df_bow)



       aa  aba  abandon  abandoned  abandonment  abased  abatement  abb  \
0       0    0        0          0            0       0          0    0   
1       0    0        0          0            0       0          0    0   
2       0    0        0          0            0       0          0    0   
3       0    0        0          0            0       0          0    0   
4       0    0        0          0            0       0          0    0   
...    ..  ...      ...        ...          ...     ...        ...  ...   
17635   0    0        0          0            0       0          0    0   
17636   0    0        0          0            0       0          0    0   
17637   0    0        0          0            0       0          0    0   
17638   0    0        0          0            0       0          0    0   
17639   0    0        0          0            0       0          0    0   

       abbas  abbey  ...  zinc  zinfandel  zing  zip  zipper  zonal  zone  \
0          0      0  .

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report
import joblib


y = df['Class_Label']

# Veri setini eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(df_bow, y, test_size=0.2, random_state=42)

In [ ]:
# Logistic regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report
import joblib


y = df['Class_Label']

# Veri setini eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(df_bow, y, test_size=0.2, random_state=42)

# Lojistik regresyon modelini eğit
model = LogisticRegression(max_iter=10000)  
model.fit(X_train, y_train)

# Test kümesi üzerinde tahminlerde bulun
predictions = model.predict(X_test)

# Modeli değerlendir
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

print("Sınıflandırma Raporu:")
print(classification_report(y_test, predictions))


joblib.dump(model, './models/bow_lr_trained_model.pkl')
joblib.dump(vectorizer, './models/lr_vectorizer.pkl')


Accuracy: 0.9705215419501134
Precision: 0.9708492507431445
Recall: 0.9705215419501134
Sınıflandırma Raporu:
              precision    recall  f1-score   support

  Fraudulent       0.98      0.92      0.95      1041
      Normal       0.97      0.99      0.98      2487

    accuracy                           0.97      3528
   macro avg       0.97      0.95      0.96      3528
weighted avg       0.97      0.97      0.97      3528



['./models/lr_vectorizer.pkl']

In [ ]:
#RF

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, classification_report

# Rastgele Orman modelini başlat
rf_model = RandomForestClassifier()

# Rastgele Orman modelini eğit
rf_model.fit(X_train, y_train)

# Test setinde tahminlerde bulun
rf_predictions = rf_model.predict(X_test)

# Modeli değerlendir
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions, average='weighted')
rf_recall = recall_score(y_test, rf_predictions, average='weighted')

print("Random Forest Classifier:")
print(f"Accuracy: {rf_accuracy}")
print(f"Precision: {rf_precision}")
print(f"Recall: {rf_recall}")

print("Classification Report:")
print(classification_report(y_test, rf_predictions))

# Eğitilmiş modeli kaydet
joblib.dump(rf_model, './models/bow_rf_trained_model.pkl')


Random Forest Classifier:
Accuracy: 0.9750566893424036
Precision: 0.9752482665059014
Recall: 0.9750566893424036
Classification Report:
              precision    recall  f1-score   support

  Fraudulent       0.98      0.93      0.96      1041
      Normal       0.97      0.99      0.98      2487

    accuracy                           0.98      3528
   macro avg       0.98      0.96      0.97      3528
weighted avg       0.98      0.98      0.97      3528



['./models/bow_rf_trained_model.pkl']

In [ ]:
# NB

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report

# Naive Bayes modelini başlat
nb_model = MultinomialNB()

# Naive Bayes modelini eğit
nb_model.fit(X_train, y_train)

# Test seti üzerinde tahminler yap
nb_predictions = nb_model.predict(X_test)

# Modeli değerlendir
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_precision = precision_score(y_test, nb_predictions, average='weighted')
nb_recall = recall_score(y_test, nb_predictions, average='weighted')

print("Naive Bayes Classifier:")
print(f"Accuracy: {nb_accuracy}")
print(f"Precision: {nb_precision}")
print(f"Recall: {nb_recall}")

print("Classification Report:")
print(classification_report(y_test, nb_predictions))

# Eğitilmiş modeli kaydet
joblib.dump(nb_model, './models/bow_nb_trained_model.pkl')


Naive Bayes Classifier:
Accuracy: 0.9623015873015873
Precision: 0.9628056385597717
Recall: 0.9623015873015873
Classification Report:
              precision    recall  f1-score   support

  Fraudulent       0.98      0.89      0.93      1041
      Normal       0.96      0.99      0.97      2487

    accuracy                           0.96      3528
   macro avg       0.97      0.94      0.95      3528
weighted avg       0.96      0.96      0.96      3528



['./models/bow_nb_trained_model.pkl']

In [4]:
# SVM

from sklearn.svm import SVC

# SVM modelini başlat
svm_model = SVC()

# SVM modelini eğit
svm_model.fit(X_train, y_train)

# Test seti üzerinde tahminler yap
svm_predictions = svm_model.predict(X_test)

# Modeli değerlendir

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions, average='weighted')
svm_recall = recall_score(y_test, svm_predictions, average='weighted')

print("Support Vector Machine Classifier:")
print(f"Accuracy: {svm_accuracy}")
print(f"Precision: {svm_precision}")
print(f"Recall: {svm_recall}")

print("Classification Report:")
print(classification_report(y_test, svm_predictions))
